<a href="https://colab.research.google.com/github/talhaanwarch/Plant-Leaf-Disease-Classification/blob/master/transfer_learning_interdomain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### download data

### install efficinetnet model

In [1]:
!pip install efficientnet

In [2]:
!wget https://data.mendeley.com/datasets/3f83gxmv57/2/files/53398b67-6f0e-4a67-8384-e2b574b2ebf4/Citrus.zip

--2020-03-31 11:50:51--  https://data.mendeley.com/datasets/3f83gxmv57/2/files/53398b67-6f0e-4a67-8384-e2b574b2ebf4/Citrus.zip
Resolving data.mendeley.com (data.mendeley.com)... 162.159.130.86, 162.159.133.86, 2606:4700:7::a29f:8256, ...
Connecting to data.mendeley.com (data.mendeley.com)|162.159.130.86|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/ad404a4d-cedf-4830-8a11-0fc204ce8acb [following]
--2020-03-31 11:50:53--  https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/ad404a4d-cedf-4830-8a11-0fc204ce8acb
Resolving md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)... 52.218.24.35
Connecting to md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)|52.218.24.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66977095 (64M) [

### unzip donwloaded dataset

In [3]:
!unzip /content/Citrus.zip

Archive:  /content/Citrus.zip
   creating: Citrus/
   creating: Citrus/Fruits/
   creating: Citrus/Fruits/Black spot/
  inflating: Citrus/Fruits/Black spot/Black spot (1).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (10).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (11).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (12).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (13).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (14).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (15).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (16).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (17).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (18).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (19).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (2).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (3).jpg  
  inflating: Citrus/Fruits/Black spot/Black spot (4).jpg  
  inflating: Citrus/Fruits/Black spot/Black sp

## import libraries

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


In [0]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import tensorflow as tf

## data augmentation 

In [0]:
from keras_preprocessing.image import ImageDataGenerator

data_gen= ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest',
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    validation_split=0.3)

In [0]:
img_shape=300
batch_size=16

# Leaves

In [8]:
#load the training data
leaves_train_generator = data_gen.flow_from_directory(
    '/content/Citrus/Leaves',
    target_size=(img_shape,img_shape),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True)


Found 429 images belonging to 5 classes.


In [9]:
#load the training data
val_suffle=False
leaves_val_generator = data_gen.flow_from_directory(
    '/content/Citrus/Leaves',
    target_size=(img_shape,img_shape),
    batch_size=16,
    class_mode='categorical',
    subset='validation',shuffle=val_suffle)#shuffle should be False in validation
    #https://stackoverflow.com/a/47439789/11170350


Found 180 images belonging to 5 classes.


In [10]:
leaves_train_generator.next()[0].shape,leaves_train_generator.next()[1].shape

((16, 300, 300, 3), (16, 5))

In [11]:
leaves_val_generator.next()[0].shape,leaves_val_generator.next()[1].shape

((16, 300, 300, 3), (16, 5))

### Base model

In [12]:
import efficientnet.tfkeras as efn
leaves_model =efn.EfficientNetB4(weights ='noisy-student', include_top=False, input_shape = (img_shape,img_shape,3))


71680000/71678424 [==============================] - 6s 0us/step


In [0]:
x = leaves_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
predictions = Dense(5, activation="softmax")(x)

In [0]:
leaves_model = Model(inputs=leaves_model.input, outputs=predictions)

In [0]:
def custom_loss(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)

In [0]:
leaves_model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

In [17]:

trainable_count = np.sum([K.count_params(w) for w in leaves_model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in leaves_model.non_trainable_weights])

print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

Total params: 17,911,901
Trainable params: 17,786,701
Non-trainable params: 125,200


In [0]:
results = leaves_model.fit(leaves_train_generator,epochs=15,
                              steps_per_epoch=leaves_train_generator.n/batch_size,
                              validation_data=leaves_val_generator,
                             validation_steps=leaves_val_generator.n/batch_size,
                              callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=2, min_lr=0.000001)])

Epoch 1/15
27/26 [==============================] - 46s 2s/step - loss: 1.0846 - accuracy: 0.6457 - val_loss: 1.3474 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 2/15
27/26 [==============================] - 40s 1s/step - loss: 0.7906 - accuracy: 0.8368 - val_loss: 1.1324 - val_accuracy: 0.5944 - lr: 0.0010
Epoch 3/15
27/26 [==============================] - 40s 1s/step - loss: 0.6825 - accuracy: 0.8811 - val_loss: 1.0418 - val_accuracy: 0.6167 - lr: 0.0010
Epoch 4/15
27/26 [==============================] - 40s 1s/step - loss: 0.6747 - accuracy: 0.8741 - val_loss: 1.8749 - val_accuracy: 0.5333 - lr: 0.0010
Epoch 5/15
27/26 [==============================] - 40s 1s/step - loss: 0.7097 - accuracy: 0.8741 - val_loss: 1.1298 - val_accuracy: 0.7056 - lr: 0.0010
Epoch 6/15
27/26 [==============================] - 40s 1s/step - loss: 0.6162 - accuracy: 0.9068 - val_loss: 0.9661 - val_accuracy: 0.7722 - lr: 3.0000e-04
Epoch 7/15
27/26 [==============================] - 40s 1s/step - loss: 0.6065

In [0]:

plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('leaves_model accuracy ')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:

plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('leaves_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
leaves_model.evaluate(leaves_val_generator,steps=leaves_val_generator.n//batch_size)

In [0]:
leaves_val_generator.reset()
leaves_model.evaluate(leaves_val_generator,steps=leaves_val_generator.n//batch_size)

In [0]:
leaves_val_generator.reset()
y_pred=leaves_model.predict(leaves_val_generator,steps=leaves_val_generator.n/batch_size,verbose=1)

In [0]:
if val_suffle==True:
  per = np.random.permutation(leaves_val_generator.n)
  leaves_val_generator.index_array = per
  y_true = leaves_val_generator.classes[per]
else: 
  y_true = leaves_val_generator.classes

In [0]:
y_pred=np.argmax(y_pred, axis=1)

In [0]:
y_pred.shape,y_true.shape

In [0]:
from sklearn.metrics import classification_report,accuracy_score,f1_score
print(accuracy_score(y_true,y_pred))
print(f1_score(y_true,y_pred,average='macro'))

In [0]:
class_label=list(leaves_train_generator.class_indices.keys())
print(classification_report(y_true, y_pred,target_names=class_label))#

In [0]:
leaves_model.save('leave_model.h5')

# Fruit

In [0]:
#load the training data
fruits_train_generator = data_gen.flow_from_directory(
    '/content/Citrus/Fruits',
    target_size=(img_shape,img_shape),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True)


In [0]:
#load the training data
val_suffle=False
fruits_val_generator = data_gen.flow_from_directory(
    '/content/Citrus/Fruits',
    target_size=(img_shape,img_shape),
    batch_size=16,
    class_mode='categorical',
    subset='validation',shuffle=val_suffle)#shuffle should be False in validation
    #https://stackoverflow.com/a/47439789/11170350


In [0]:
fruits_train_generator.next()[0].shape,fruits_train_generator.next()[1].shape

In [0]:
fruits_val_generator.next()[0].shape,fruits_val_generator.next()[1].shape

### Base model

In [0]:
import efficientnet.tfkeras as efn
fruit_model =efn.EfficientNetB4(weights ='noisy-student', include_top=False, input_shape = (img_shape,img_shape,3))


In [0]:
x = fruit_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
predictions = Dense(5, activation="softmax")(x)

In [0]:
fruit_model = Model(inputs=fruit_model.input, outputs=predictions)

In [0]:
def custom_loss(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)

In [0]:
fruit_model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

In [0]:

trainable_count = np.sum([K.count_params(w) for w in fruit_model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in fruit_model.non_trainable_weights])

print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

In [0]:
results = fruit_model.fit(fruits_train_generator,epochs=15,
                              steps_per_epoch=fruits_train_generator.n/batch_size,
                              validation_data=fruits_val_generator,
                             validation_steps=fruits_val_generator.n/batch_size,
                              callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=2, min_lr=0.000001)])

In [0]:

plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('fruit_model accuracy ')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:

plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('fruit_model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
fruit_model.evaluate(fruits_val_generator,steps=fruits_val_generator.n//batch_size)

In [0]:
fruits_val_generator.reset()
fruit_model.evaluate(fruits_val_generator,steps=fruits_val_generator.n//batch_size)

In [0]:
fruits_val_generator.reset()
y_pred=fruit_model.predict(fruits_val_generator,steps=fruits_val_generator.n/batch_size,verbose=1)

In [0]:
if val_suffle==True:
  per = np.random.permutation(fruits_val_generator.n)
  fruits_val_generator.index_array = per
  y_true = fruits_val_generator.classes[per]
else: 
  y_true = fruits_val_generator.classes

In [0]:
y_pred=np.argmax(y_pred, axis=1)

In [0]:
y_pred.shape,y_true.shape

In [0]:
from sklearn.metrics import classification_report,accuracy_score,f1_score
print(accuracy_score(y_true,y_pred))
print(f1_score(y_true,y_pred,average='macro'))

In [0]:
class_label=list(fruits_train_generator.class_indices.keys())
print(classification_report(y_true, y_pred,target_names=class_label))#

# Model Transfer
Leave model to Fruit model

In [0]:
import efficientnet.tfkeras as efn
leave2fruit =efn.EfficientNetB4(weights =None, include_top=False, input_shape = (img_shape,img_shape,3))


In [0]:
x = leave2fruit.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
predictions = Dense(5, activation="softmax")(x)

In [0]:
leave2fruit = Model(inputs=leave2fruit.input, outputs=predictions)

In [0]:
def custom_loss(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)

In [0]:
leave2fruit.compile(optimizer=Adam(0.0001), loss=custom_loss, metrics=['accuracy'])

In [0]:
leave2fruit.load_weights('leave_model.h5')

In [0]:

trainable_count = np.sum([K.count_params(w) for w in leave2fruit.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in leave2fruit.non_trainable_weights])

print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

In [0]:
results = leave2fruit.fit(fruits_train_generator,epochs=15,
                              steps_per_epoch=fruits_train_generator.n/batch_size,
                              validation_data=fruits_val_generator,
                             validation_steps=fruits_val_generator.n/batch_size,
                              callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.000001)])

In [0]:

plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('leave2fruit accuracy ')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:

plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('leave2fruit loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
leave2fruit.evaluate(fruits_val_generator,steps=fruits_val_generator.n//batch_size)

In [0]:
fruits_val_generator.reset()
leave2fruit.evaluate(fruits_val_generator,steps=fruits_val_generator.n//batch_size)

In [0]:
fruits_val_generator.reset()
y_pred=leave2fruit.predict(fruits_val_generator,steps=fruits_val_generator.n/batch_size,verbose=1)

In [0]:
if val_suffle==True:
  per = np.random.permutation(fruits_val_generator.n)
  fruits_val_generator.index_array = per
  y_true = fruits_val_generator.classes[per]
else: 
  y_true = fruits_val_generator.classes

In [0]:
y_pred=np.argmax(y_pred, axis=1)

In [0]:
y_pred.shape,y_true.shape

In [0]:
from sklearn.metrics import classification_report,accuracy_score,f1_score
print(accuracy_score(y_true,y_pred))
print(f1_score(y_true,y_pred,average='macro'))

In [0]:
class_label=list(fruits_train_generator.class_indices.keys())
print(classification_report(y_true, y_pred,target_names=class_label))#

# Model Transfer
Leave model to Fruit model